In [7]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense

In [8]:
model = Sequential()
model.add(ResNet50(include_top = False, weights = "imagenet", pooling = "avg", input_shape = (150,150,3)))
model.add(Dense(6,activation = "softmax"))
model.layers[0].trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

94658560/94653016 [==============================] - 804s 8us/step


In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12294     
Total params: 23,600,006
Trainable params: 12,294
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from albumentations import *
def aug_image(image):
    aug = Compose([
        HorizontalFlip(p=0.5),
        ShiftScaleRotate(shift_limit=0.06, scale_limit=0.2, rotate_limit=10, p=0.5),
        RandomBrightnessContrast(p = 0.5)
    ],p=0.5)
    image = aug(image = image)['image']
    return image
imagegen = ImageDataGenerator(preprocessing_function=aug_image,rescale=1./255)
imagegen_ = ImageDataGenerator(preprocessing_function=None,rescale=1./255)
train = imagegen.flow_from_directory("data/intel-image-classification/seg_train/seg_train",target_size=(150, 150),batch_size=256)
val = imagegen_.flow_from_directory("data/intel-image-classification/seg_test/seg_test",target_size=(150, 150),batch_size=128)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [14]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [16]:
epochs = 10

In [ ]:
history = model.fit_generator(train, epochs=epochs, steps_per_epoch = 50, validation_data=val, verbose = 1, callbacks=[learning_rate_reduction])


Epoch 1/10
 4/50 [=>............................] - ETA: 38:10 - loss: 2.1065 - accuracy: 0.2031

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

fig, ax = plt.subplots(2,1, figsize=(8, 6))
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)